# Riak
![png](https://upload.wikimedia.org/wikipedia/en/8/8e/Riak_distributed_NoSQL_key-value_data_store_logo.png)

## Conexión a Riak

In [1]:
import riak

In [2]:
cliente = riak.RiakClient()
print "¿Está Riak disponible?", cliente.ping()

¿Está Riak disponible? True


## Insertar valores

In [3]:
bucket = cliente.bucket("bucket_name")

In [4]:
key1 = "key_one"
val1 = 1
key = bucket.new(key1, data = val1)
key.store()

In [8]:
key2 = "key_two"
val2 = "Dos"
key = bucket.new(key2, data = val2)
key.store()

In [11]:
key3 = "key_three"
val3 = {
    "val1" : 1,
    "val2" : "Dos",
    "val3" : [1, 2, 3, 4, 5],
    
}

In [12]:
key = bucket.new(key3, data = val3)
key.store()

## Insertando imágenes

In [13]:
image_bucket = cliente.bucket('images')

photo = image_bucket.new_from_file('image_key', './data/demo.image.jpg')
photo.store()

## Recuperando información

In [14]:
from pprintpp import pprint as pp
bucket = cliente.bucket("bucket_name")

print "key_one:", bucket.get("key_one").data
print "key_two:", bucket.get("key_two").data
data3 = bucket.get("key_three").data

print "key_three: "
pp(data3)

key_one: 1
key_two: Dos
key_three: 
{u'val1': 1, u'val2': u'Dos', u'val3': [1, 2, 3, 4, 5]}


In [15]:
!curl http://localhost:8098/riak/bucket_name/key_one

1

In [16]:
!curl http://localhost:8098/riak/bucket_name/key_two

"Dos"

In [17]:
!curl  http://localhost:8098/riak/bucket_name/key_three

{"val3": [1, 2, 3, 4, 5], "val2": "Dos", "val1": 1}

http://localhost:8098/riak/images/image_key

## Utilizando índices

In [18]:
bucket = cliente.bucket("ejemplo_indices")

for i in range(1, 10):
    key = `i`
    valor_string = "Valor " + `i`
    valor = {"id1" : key,
             "id2" : valor_string}

    riak_key = bucket.new(key, data = valor)
    riak_key.add_index('index_id2_bin', valor_string)
    riak_key.store()
    
    print "key", key, ":", valor


key 1 : {'id2': 'Valor 1', 'id1': '1'}
key 2 : {'id2': 'Valor 2', 'id1': '2'}
key 3 : {'id2': 'Valor 3', 'id1': '3'}
key 4 : {'id2': 'Valor 4', 'id1': '4'}
key 5 : {'id2': 'Valor 5', 'id1': '5'}
key 6 : {'id2': 'Valor 6', 'id1': '6'}
key 7 : {'id2': 'Valor 7', 'id1': '7'}
key 8 : {'id2': 'Valor 8', 'id1': '8'}
key 9 : {'id2': 'Valor 9', 'id1': '9'}


In [19]:
keys = bucket.get_index("index_id2_bin", 'Valor 5')
for key in keys.results:
    print key

5


## Clave única

In [20]:
import time
import uuid

bucket_estado = cliente.bucket('status')

post_uuid = uuid.uuid1().hex
estado_object = riak.RiakObject(cliente, bucket_estado, post_uuid)

# Añadimos los datos que queremos almacenar.
estado_object.data = {
    'message': 'Message',
    'created': time.time(),
    'is_public': True
}

# Establecemos el content type.
estado_object.content_type = 'application/json'

# Queremos que se realice un enconding JSON sobre el valor
estado_object._encode_data = True

# Again, make sure you save it.
estado_object.store()

print post_uuid

3c78bb488c2911e58768080027a4c0a1


http://localhost:8098/riak/status/0222478086dd11e59398080027a4c0a1

In [22]:
!curl -i http://localhost:8098/riak/status/3c78bb488c2911e58768080027a4c0a1












{"is_public": true, "message": "Message", "created": 1447654444.076801}

## Listado de Buckets

In [23]:
for bucket in cliente.get_buckets():
        print('Bucket %s' % bucket )

Bucket <RiakBucket 'users'>
Bucket <RiakBucket 'tweets'>
Bucket <RiakBucket 'food'>
Bucket <RiakBucket 'images'>
Bucket <RiakBucket 'hashtags'>
Bucket <RiakBucket 'status'>
Bucket <RiakBucket 'ejemplo_indices'>
Bucket <RiakBucket 'people'>
Bucket <RiakBucket 'bucket_name'>


## Listado de Claves de un Bucket

In [24]:
for keys in bucket_estado.stream_keys():
    for key in keys:
        print('Key %s' % key )

Key 3c78bb488c2911e58768080027a4c0a1
